# Determining Rebalance Day and Hour
## By: Carter Carlson
### 2020.04.05

### Description
Fart Set will rebalance once per week at the same day and hour.  Instead of picking an arbitrary time to rebalance on, let's find:

* The day of week with the least price volatility
* On that day, the hour with the least price volatility

In [2]:
import pandas as pd

eth_1h = pd.read_csv('../data/ETH.csv')

print(f"Start date: {eth_1h['date'].iat[0]}")
print(f"End date: {eth_1h['date'].iat[-1]}")

Start date: 2019-01-01 00:00:00
End date: 2019-11-28 23:00:00


In [3]:
eth_1h.head()

,date,open,high,low,close,volume
0,2019-01-01 00:00:00,132.74,133.50,132.67,133.36,6204.28401
1,2019-01-01 01:00:00,133.36,134.30,133.01,133.56,10214.95098
2,2019-01-01 02:00:00,133.56,135.30,133.19,134.52,16342.20664
3,2019-01-01 03:00:00,134.55,134.78,133.55,133.70,14895.35710
4,2019-01-01 04:00:00,133.72,134.14,133.66,133.86,6388.10191


In [4]:
# Group ETH into 1d candles
# https://pypi.org/project/TAcharts
from TAcharts.utils import group_candles

eth_1d = group_candles(eth_1h, n=24)
eth_1d.head()

,date,open,high,low,close,volume
0,2019-01-01 00:00:00,132.74,140.67,131.00,138.99,4.151021e+05
1,2019-01-02 00:00:00,138.91,155.37,138.30,151.93,1.018141e+06
2,2019-01-03 00:00:00,151.78,154.20,143.11,153.33,7.381434e+05
3,2019-01-04 00:00:00,153.11,159.26,144.66,154.51,8.914434e+05
4,2019-01-05 00:00:00,157.70,157.80,147.51,150.47,6.645218e+05


### Daily Volatility

In [5]:
def calc_pct_change(high, low, _open):
    return (high - low) / _open

In [6]:
eth_1d['pct_change'] = calc_pct_change(eth_1d['high'], eth_1d['low'], eth_1d['open'])
eth_1d['weekday'] = pd.DatetimeIndex(eth_1d['date']).day_name()

daily_volatility = eth_1d.groupby('weekday')['pct_change'].mean().sort_values(ascending=False)
print(daily_volatility)

weekday
Tuesday      0.070740
Wednesday    0.066079
Sunday       0.063304
Monday       0.062416
Friday       0.061621
Thursday     0.059621
Saturday     0.052946
Name: pct_change, dtype: float64


#### Saturday has the lowest average daily volatility.

### Hourly Volatility

In [7]:
eth_1h['pct_change'] = calc_pct_change(eth_1h['high'], eth_1h['low'], eth_1h['open'])
eth_1h['hour'] = pd.DatetimeIndex(eth_1h['date']).hour

# Only take volatility from Saturday
eth_1h_saturday = eth_1h[pd.DatetimeIndex(eth_1h['date']).day_name() == 'Saturday']

hourly_volatility = eth_1h.groupby('hour')['pct_change'].mean().sort_values(ascending=False)
print(hourly_volatility)

hour
11    0.015472
19    0.014008
10    0.013859
9     0.013478
18    0.013072
17    0.012644
8     0.012570
15    0.012349
7     0.012239
16    0.011714
5     0.011542
12    0.011506
4     0.011314
3     0.011192
6     0.011139
14    0.010986
20    0.010895
13    0.010425
23    0.010207
22    0.010100
21    0.010055
2     0.010032
0     0.009698
1     0.009190
Name: pct_change, dtype: float64


#### On Saturday, hour 1 has the lowest average volatility.

## Conclusion: Fart Set will rebalance every Saturday at 01:00.